<a href="https://colab.research.google.com/github/masonnystrom/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/LS_DS_212_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [222]:
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# create interst level numeric 
df['interest_level_numeric'] = df['interest_level'].replace({'high':1,'medium':2,'low':3})

In [224]:
df.dtypes

bathrooms                 float64
bedrooms                    int64
created                    object
description                object
display_address            object
latitude                  float64
longitude                 float64
price                       int64
street_address             object
interest_level             object
elevator                    int64
cats_allowed                int64
hardwood_floors             int64
dogs_allowed                int64
doorman                     int64
dishwasher                  int64
no_fee                      int64
laundry_in_building         int64
fitness_center              int64
pre-war                     int64
laundry_in_unit             int64
roof_deck                   int64
outdoor_space               int64
dining_room                 int64
high_speed_internet         int64
balcony                     int64
swimming_pool               int64
new_construction            int64
terrace                     int64
exclusive     

In [0]:
from sklearn.model_selection import train_test_split

In [226]:
# train test split
train, test = train_test_split(df, test_size=0.2)
print(test.shape)
train.shape

(9764, 35)


(39053, 35)

In [227]:
# Baseline
train['price'].mean()

3575.758430850383

In [0]:
# arrange y vectors 
target = 'price'
base_y_train = train[target]
base_y_test = test[target]


In [229]:
# y train baseline 
mean_baseline = base_y_train.mean()
print("Mean Baseline using ('0 Features)")

Mean Baseline using ('0 Features)


In [230]:
# Baseline train error
from sklearn.metrics import mean_absolute_error
base_y_pred = [mean_baseline] * len(base_y_train)
mae = mean_absolute_error(base_y_train, base_y_pred)
print(f'Train Error (Price of Rent in NYC): {mae:.2f} U.S dollars')

Train Error (Price of Rent in NYC): 1198.37 U.S dollars


In [231]:
# Baseline Test Error
base_y_pred = [mean_baseline] * len(base_y_test)
mae = mean_absolute_error(base_y_test, base_y_pred)
print(f'Test Error (Price of rent in NYC): {mae:.2f} U.S dollars')

Test Error (Price of rent in NYC): 1208.88 U.S dollars


In [0]:
# Feature engineering 
import plotly.express as px

In [233]:
px.scatter(train,'interest_level_numeric', 'price', trendline='ols')

In [0]:
# Model testing, import model 
from sklearn.linear_model import LinearRegression

In [0]:
# instantiate the class 
model = LinearRegression()

In [236]:
# arranging the x features
features = ['interest_level_numeric', 'bedrooms']
x_train = train[features]
x_test = test[features]
print(f'Linear Regression, dependent on: {features}')


Linear Regression, dependent on: ['interest_level_numeric', 'bedrooms']


In [237]:
print(x_test.shape)
print(x_train.shape)

(9764, 2)
(39053, 2)


In [238]:
# fit the model 
model.fit(x_train, base_y_train)
y_pred_train = model.predict(x_train)
mae = mean_absolute_error(base_y_train, y_pred_train)
print(f'Train Error: {mae:.2f}')

Train Error: 934.26


In [239]:
# apply the model to new data
y_pred = model.predict(x_test)
mae = mean_absolute_error(base_y_test, y_pred)
print(f'Test Error: {mae:.2f}')

Test Error: 943.31


In [240]:
# intercepts and coefficients 
model.intercept_, model.coef_

(581.2355120907196, array([636.76238057, 866.25890027]))

In [241]:
beta0 = model.intercept_
beta1, beta2 = model.coef_
print(f'y = {beta0} + {beta1}x1 + {beta2}x2')

y = 581.2355120907196 + 636.7623805701498x1 + 866.258900271871x2


In [0]:
from sklearn.metrics import mean_squared_error, r2_score

In [243]:
print(y_pred.shape)
print(base_y_test.shape)

print(x_test.shape)
print(x_train.shape)

print(base_y_train.shape)
print(y_pred_train.shape)


(9764,)
(9764,)
(9764, 2)
(39053, 2)
(39053,)
(39053,)


In [246]:
# Train  MSE, RMSE, MAE, and R2
mse = mean_squared_error(base_y_train, y_pred_train, squared=False)
rmse = np.sqrt(mse)
mae = mean_absolute_error(base_y_train, y_pred_train)
r2 = r2_score(base_y_train, y_pred_train)
print('Mean Squared Error:', mse)
print('Root Mean Squared Error:', rmse)
print('Mean Absolute Error:', mae)
print('R^2:', r2)

Mean Squared Error: 1430.0547081638547
Root Mean Squared Error: 37.81606415485164
Mean Absolute Error: 934.2624697330344
R^2: 0.3372039626333688


In [217]:
# Test for MSE, RMSE, MAE, and R2
mse = mean_squared_error(base_y_test, y_pred, squared=False)
rmse = np.sqrt(mse)
mae = mean_absolute_error(base_y_test, y_pred)
r2 = r2_score(base_y_test, y_pred)
print('Mean Squared Error:', mse)
print('Root Mean Squared Error:', rmse)
print('Mean Absolute Error:', mae)
print('R^2:', r2)

Mean Squared Error: 1442.9682920725436
Root Mean Squared Error: 37.98642247004242
Mean Absolute Error: 929.5413507788808
R^2: 0.3395502437779261
